In [24]:
import json
import pandas as pd
import tradingeconomics as te

In [25]:
te.login('kkuokmtiaotfc39:suaiyzpd9avf8so')

'You are logged in as a kkuokmtiaotfc39:suaiyzpd9avf8so'

In [26]:
raw_data = te.getHistoricalData(country=['Bangladesh', 'China', 'Japan', 'Israel', 'India', 'South Korea'], indicator='Wages', initDate='2015-03-01', output_type='df')

In [27]:
country_df = raw_data.loc[raw_data['Country'] == 'South Korea']
country_df = pd.DataFrame(country_df)

country_df.sort_values(by='DateTime', inplace=True, ascending=True)
average_wage = country_df.iloc[-1,]
average_wage

Country                         South Korea
Category                              Wages
DateTime                2022-03-31T00:00:00
Value                             4505228.0
Frequency                         Quarterly
HistoricalDataSymbol           SOUTHKOREWAG
LastUpdate              2022-06-04T10:03:00
Name: 207, dtype: object

In [54]:
def func_wage(country, ticker):
    raw_data = te.getHistoricalData(country=country, indicator='Wages', initDate='2015-03-01', output_type='df')    
    country_name = raw_data.loc[raw_data['Country'] == country]
    country_df = pd.DataFrame(country_name)

    country_df.sort_values(by='DateTime', inplace=True, ascending=True)
    average_wage = country_df.iloc[-1,]
    
    average_wage = pd.DataFrame(average_wage).T
    
    # Convert Currency
    average_wage['Value'] = average_wage['Value']/Convert_Curr(country, ticker)
    
    # Convert Yearly Freq
    average_wage = Convert_Yearly(country, average_wage)
    
    average_wage['Value'] = average_wage['Value'].values[0].round(1)
    
    return average_wage

In [55]:
def truncate(number, decimals=0):
    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [56]:
def Convert_Curr(country, ticker):
    currency = te.getCurrencyCross(cross = ticker, output_type = 'df')
    currency = currency.loc[currency['Country'] == country]
    output = currency['Close']
    output = output.values[0]
    # output = truncate(output, 2)
    return output

In [57]:
#Convert yearly wage to monthly
def Convert_Yearly(country, average_wage):
    raw_data = te.getIndicatorData(country=country, indicators='wages', output_type='df')
    unit = raw_data['Unit'].values[0]
    unit = unit.split("/")
    unit = unit[1]
    
    if unit == 'Year':
        average_wage['Value'] = (average_wage['Value']/12)
    else:
        return average_wage
        
    average_wage['Value'] = average_wage['Value'].values[0].round(1)
    return average_wage

In [58]:
with open('country.json', 'r') as f:
    data = json.load(f)
    
data = pd.DataFrame(data['data'])

In [59]:
data

,name,ticker,lon,lat
0,China,CNY,104.1954,35.8617
1,Japan,JPY,138.2529,36.2048
2,Israel,ILS,34.8516,31.0461
3,Malaysia,MYR,101.9758,4.2105
4,South Korea,KRW,127.7669,35.9078
5,Armenia,AMD,45.0382,40.0691
6,Azerbaijan,AZN,47.5769,40.1431
7,Bahrain,BHD,50.5577,26.0667
8,Bangladesh,BDT,90.3563,23.6850
9,Georgia,GEL,82.9001,32.1656


In [60]:
length = len(data.index)

Average_Wage_in_Asia = pd.DataFrame()
for i in range(length):
    w = func_wage(data['name'].values[i], data['ticker'].values[i])
    Average_Wage_in_Asia = pd.concat([Average_Wage_in_Asia, w])

In [61]:
Wage = Average_Wage_in_Asia
Wage

,Country,Category,DateTime,Value,Frequency,HistoricalDataSymbol,LastUpdate
6,China,Wages,2021-12-31T00:00:00,1335.0,Yearly,CHINAWAG,2022-06-04T10:01:00
84,Japan,Wages,2022-03-31T00:00:00,2472.1,Monthly,JAPANWAG,2022-05-14T10:02:00
84,Israel,Wages,2022-03-31T00:00:00,3608.5,Monthly,ISRAELWAG,2022-06-04T10:06:00
5,Malaysia,Wages,2020-12-31T00:00:00,667.6,Yearly,MALAYSIAWAG,2021-07-20T14:21:00
28,South Korea,Wages,2022-03-31T00:00:00,3593.9,Quarterly,SOUTHKOREWAG,2022-06-04T10:03:00
84,Armenia,Wages,2022-03-31T00:00:00,517.7,Monthly,ARMENIAWAG,2022-05-30T07:02:00
84,Azerbaijan,Wages,2022-03-31T00:00:00,477.8,Monthly,AZERBAIJANWAG,2022-05-21T10:11:00
26,Bahrain,Wages,2021-09-30T00:00:00,2077.5,Quarterly,BAHRAINWAG,2022-02-08T19:08:00
1,Bangladesh,Wages,2017-12-31T00:00:00,144.2,Yearly,BANGLADESHWAG,2018-04-17T11:13:00
27,Georgia,Wages,2021-12-31T00:00:00,503.0,Quarterly,GEORGIAWAG,2022-03-19T10:07:00


In [62]:
value = []
value = Average_Wage_in_Asia['Value'].values
wage_in_float = pd.DataFrame(value).astype(float)

In [63]:
Wage['Value'] = '$' + Wage['Value'].astype(str)
data['wage'] = Wage['Value'].values
data['wage_in_float'] = wage_in_float

In [64]:
scale = {'Color': ['green', 'blue', 'orange', 'red'], 'Wage': ['above $2000', ' $1000 to $1,999', '$500 to $999', 'below $500']}  
scale = pd.DataFrame(scale)
scale

def color_range(wage):
    if wage < 500:
        color = "red"
    elif (wage >= 500) & (wage <= 999):
        color = "orange"
    elif (wage >= 1000) & (wage <= 1999):
        color = "blue"
    elif (wage>=2000):
        color = "green"
    
    return color

In [65]:
import folium

m = folium.Map(location=[34.0479, 100.6197], zoom_start=3)

folium.LayerControl().add_to(m)

for i in range(len(data.index)):
    folium.Marker(location=[data['lat'].values[i], data['lon'].values[i]],
             icon = folium.DivIcon(
                 html="""
                 <div style='color: white;background-color:"""+color_range(data['wage_in_float'].values[i])+"""; width: 90px; padding: 2px; text-align: center'>
                     """+ data['name'].values[i] +" "+data['wage'].values[i]+"""
                 </div>
                 """,
                 class_name = "mapText2"
             )).add_to(m)

m